<p style = "font-size: 37px">Phases</p>
<br>
<ul style = "font-size : 25px;">
<li ><a href = "#Loading the data">Loading the data</a></li>
<br/>    
<li><a href = "#Data preprocessing">Data preprocessing</a></li>
<br/>      
<li><a href = "#Model Building">Model Building</a></li>       
</ul>   

# Loading the Data
<p id = "Loading the data"></p>

In [ ]:
import re                                                  # use regular expression to deal with text
from nltk.tokenize import sent_tokenize, word_tokenize     # for word and sentence tokenization
from nltk.corpus import stopwords                          # to help removing stopping words
from nltk.stem import WordNetLemmatizer                    # to lemmatize the words
from string import punctuation                             # to remove punctuation like ? ! . etc
import io                                                  # to read embeddings from text file
import math                                                # to get mathmatical functions sin cos etc
import pandas as pd                                        # to deal with dataframes    
import numpy as np                                         # to deal with numbers and dataframes  
import tensorflow as tf                                    # the main package to build the transformer
import sys                                                 # good printing tool
import pickle                                              # importing the parameters

In [ ]:
# red the data as datafram
df = pd.read_csv(r"train.csv")

In [ ]:
# let's load the embeddings as dictionary where key is the word and value it's corresponding embbeding
def load_vectors(fname):
    """
    this function takes the path of the embeddings file 'must be from fast text'
    and returns a dictionary where the words are keys and thier embeddings as values
    input :
    fname (String) ------> file path
    output :
    data (dictionary)----> dictionary to acess the embeddings with words as keys
    """
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] =  tokens[1:] 
    # add pad token as zero vector
    # we will expalain this later
    data["<PAD>"] = [0]*300    
    return data

In [ ]:
dic = load_vectors("wiki.simple.vec.text")

# Data preprocessing
<p id = "Data preprocessing"></p>

In [ ]:
def text_cleansing(text,maximum_len = 20):
    """
     This function performs the following steps:

    1. Lowercases the text
    2. Removes HTTP and WWW links
    3. Removes non-ASCII characters
    4. Removes numbers
    5. Tokenizes the text
    6. Removes stop words
    7. Lemmatizes the tokens
    8. Truncates tokens that are longer than the specified maximum length
    inputs :
    text (String) -----------> String to perform the cleaning
    maximum_len(int)--------->The maximum length of tokens to keep
    output:
    cleaned_tokens(text)----->A string containing the cleaned text

    """
    # lower case the text
    text = text.lower()
    # replace the text that have a ppatern of http link with empty link
    text = re.sub(r"(https?[\s+]?:?[\s+]?[(\w+|.)/?(\w+|.)?]+[\s]?)","",text)
    # replace the text that have a ppatern of only www. link
    text = re.sub(r"(\w+[\s+]?:[\s+]?www.\w+.\w+)","",text)
    # remove none ASCII characters like "| ¡ ¿ † ‡ ↔ ↑ ↓ • ¶"
    text = re.sub(r'[^\x00-\x7f]','',text)
    # remove numbers from the text 
    text = re.sub(r"\d","",text)
    # turn the text into list of words 
    tokens = word_tokenize(text)
    # cach the stopping words that in english
    stop_words = stopwords.words("english")
    # intialize a lemmatizer 
    lemmatizer = WordNetLemmatizer()
    # define a new list to keep the cleaned tokens
    cleaned_tokens = []
    # iterate over the tokens to remove the words that exceeds the maximum length 
    # remove stopping words
    # remove punctuation
    # lemmatize the words
    for i,token in enumerate(tokens):
        if (len(token) > maximum_len) or (token in stop_words):
            # do nothing 
            s = ""
        else:
            s = token.strip(punctuation)
            if s != "":
                cleaned_tokens.append(lemmatizer.lemmatize(s))

    return " ".join(cleaned_tokens)   

In [ ]:
# apply the claning to each row of the data
df.iloc[:,1] = df.iloc[:,1].apply(text_cleansing)

In [ ]:
def tokenizer(text):
    """
    this function truncate the sentences that exceeds 50 words
    and add paddings to senteces that is shorter than 50
    input :
    text (String) ------> text of the sentence
    output:
    tokens (list) ------> list of words 'tokens'
    """
    tokens = word_tokenize(text)  
    if len(tokens) > 50:
        tokens = tokens[:50]
        return tokens
    else :
        padding_len = 50 - len(tokens)
        tokens.extend(["<PAD>"]*padding_len)
        return tokens

In [ ]:
# apply the tokenizer to each sentence
df.iloc[:,1] = df.iloc[:,1].apply(tokenizer)

# Model Building
<p id = "Model Building"></p>

In [ ]:
class Model:
    def __init__(self, dic, n = 50, d = 300, h = 2, k = 300, c = 1200, m = 2):
        self.dic = dic
        self.n = n
        self.d = d
        self.h = h
        self.k = k
        self.c = c
        self.m = m
        self.positional_encoding      = self.positional_embeddings()
        self.encoder_parameters       = self.Encoder_parameters_intializer()
        self.head_parameters          = self.Head_parameters_intializer()
        self.parameters               = self.encoder_parameters + self.head_parameters
        
    def import_parameters(self,pname):
        with open(pname, 'rb') as file:
            # Call load method to deserialze
            self.parameters = pickle.load(file)
        self.encoder_parameters = self.parameters[:8]
        self.head_parameters    = self.parameters[8:]
        
    def input_embeddings(self, sentence):
        """
        Converts a sentence into a matrix of input embeddings of dim d x n
        d is embeddings length 
        n is the sentence length
        input :
        sentence (list) ------> list of words
        dic (dictionary)------> dictionary of embeddings
        output :
        tokens (Tensor)-------> tensor of size [d , n]

        """
        dic = self.dic
        
        tokens = []
        for word in sentence:
            if word in dic:
                tokens.append(dic[word])
            # if the word out-of-vocab use special token </s>    
            else :
                tokens.append(dic['</s>'])
        # this make sure that the array in size (d,n)        
        tokens =  np.array(tokens, dtype = np.float32).T 
        return tf.Variable(tokens)      
    def positional_embeddings(self, c = 10000):
        """
        get the positional embeddings of embeddings of size [d,n]
        inputs :
        d (int) -------> is the diemnsion of the embeddings
        n (int) -------> is the number of tokens in the sentence
        c (int) -------> hyperparameter by default 10000
        output :
        tokens (Tensor)------> positional embeddings for the tokens
        """
        d = self.d
        n = self.n
        
        tokens = []
        for position in range(n):
            tokens.append([])
            for i in range(d):
                # if i is even calculate even_pe
                if i%2 == 0:
                    even_d    = math.pow(c, i/d)
                    even_pe = math.sin(position/ even_d)
                    tokens[position].append(even_pe)
                # if i is odd calculate odd_pe    
                else:
                    odd_d    = math.pow(c, i/d)
                    odd_pe = math.cos(position/ odd_d)
                    tokens[position].append(odd_pe)            
        return tf.Variable(np.array(tokens).T, dtype = tf.float32, shape = [d,n] )
    def padding_mask(self, X):
        """
        Create mask which marks the zero padding values in the input by a 1
        this mask is used to force the model to ignore paddings
        inputs:
        X (Tensor) -------> matrix of embeddings of size [d,n]
        n (int)    -------> number of words in the sentences
        output:
        mask (Tesnor)----> mask with the same size as X
        """
        n = self.n
        
        # get boolean vector to indicate position of padding  
        mask = tf.math.equal(X, 0)
        mask = tf.reduce_all(mask, axis = 0)
        # cast the vector from boolean to float32 then rshape it as column vector
        mask = tf.cast(mask, tf.float32)
        mask = tf.reshape(mask, (1,-1))
        # repeat that column n times 
        mask = tf.concat([mask]*n, axis = 0)
        return mask
    def MHSA(self, X, U_q, U_k, V, W):
        """
        this function return multi head self attention tensor given the parameters and input matrix X
        inputs :
        X   (Tensor)----> input matrix of size [d,n]
        U_q (Tensor)---> matrix of size [h,k,d] is used to get Query matrix
        U_k (Tensor)---> matrix of size [h,k,d] is used to get Key matrix
        V   (Tensor)---> projection matrix of size [h,d,d]
        W   (Tensor)---> weight matrix with size [d,h*d] to project the concatenated heads back to X size
        h   (int)------> number of heads
        k   (int)------> dimension of Query,Key matrixes
        output :
        mhsa (Tensor)----> matrix of size [d,n]
        """
        d   = self.d
        n   = self.n
        h   = self.h
        k   = self.k
        
        Q   = tf.matmul(U_q, X)  
        K   = tf.matmul(U_k, X)
        m   = self.padding_mask(X) * -1e9 
        A   = tf.nn.softmax((tf.matmul(Q, K, transpose_a = True) + m)  / np.sqrt(k), axis= 1)
        Y    = V @ X @ A
        mhsa = W @ tf.reshape(Y, [h*d,n])
        return mhsa
    def Add_Norm(self, x1, x2):
        """
        simple function to add two tensors and then normalize the resultant tensor over their rows
        inputs :
        x1 (Tensor)-----> tensor of size [d,n]
        x2 (Tensor)-----> tensor of size [d,n]
        output:
        Normalized_X (Tenosr)-----> normalized tensor of size [d,n]
        """
        X = x1 + x2
        mean = tf.math.reduce_mean(X,axis = 1)
        mean = tf.reshape(mean, [-1,1])
        std = tf.math.reduce_std(X,axis = 1)
        std = tf.reshape(std, [-1,1]) +1e-6
        Normalized_X = (X - mean) / std
        return Normalized_X
    def feed_forward_NN(self,X, W1, W2, b1, b2):
        # c approxamatly 4 * d
        # in the paper d = 512 and c = 2048
        hidden_layer = tf.nn.relu(W1 @ X + b1)
        Dropout      = tf.nn.dropout(hidden_layer,0.5)
        output       = W2 @ Dropout + b2
        return output
    def Encoder_parameters_intializer(self):
        # this function just intialize encoder parameters
        # All the distributions have been chosen based on a lot of experiments
        h = self.h
        k = self.k
        d = self.d
        c = self.c
        
        U_q = tf.random.uniform((h,k,d), minval = 0, maxval = 1)
        U_q = tf.Variable(U_q)
        U_k = tf.random.uniform((h,k,d), minval = 0, maxval = 1)
        U_k = tf.Variable(U_k)
        V   = tf.random.uniform((h,d,d), minval = 0, maxval = 1)
        V = tf.Variable(V)
        W   = tf.random.uniform((d,h*d), minval = 0, maxval = 1)
        W = tf.Variable(W)
        W1  = tf.random.normal((c,d))
        W1 = tf.Variable(W1)
        W2  = tf.random.normal((d,c))
        W2 = tf.Variable(W2)
        b1  = tf.random.uniform((c,1))
        b1 = tf.Variable(b1)
        b2  = tf.random.uniform((d,1))
        b2 = tf.Variable(b2)
        return U_q,U_k,V,W,W1,W2,b1,b2
    
    def Head_parameters_intializer(self,c3 = 6):
        # this function just intialize Head parameters
        # All the distributions have been chosen based on a lot of experiments
        d = self.d
        
        W_c1 = tf.random.uniform((c3,d), minval = 0, maxval = 1)
        W_c1 = tf.Variable(W_c1)
        b_c1 = tf.random.uniform((c3,1))
        b_c1 = tf.Variable(b_c1)
        return W_c1,b_c1
    
    def Encoder(self, X):
        # we have described the function in details through the mathematical notation above
        U_q,U_k,V,W,W1,W2,b1,b2 = self.encoder_parameters
        X   = X + self.positional_encoding
        X    = tf.nn.dropout(X,0.1)
        mhsa = self.MHSA(X, U_q, U_k, V, W)
        X    = self.Add_Norm(X, mhsa)
        fnn  = self.feed_forward_NN(X, W1, W2, b1, b2)
        X    = self.Add_Norm(X, fnn)
        
    def Head(self, X):
        # we have described the function in details through the mathematical notation above
        W_c1,b_c1 = self.head_parameters
        output   = tf.nn.sigmoid(W_c1 @ X + b_c1)
        return output
        
    def Transformer_block(self, X):
        """
        this function combine multiple layers of the encoder to get transformer output
        inputs :
        X          (Tensor)-----> tesnor of the embeddings of size [d,n]
        parameters (tuple)------> tuple of encoder parameters
        output:
        encoder_output (Tensor)---> the output of the transformer which has the same size as X
        """
        # define one layer outside the loop then use the output as input
        # repeat the process m-1 times 
        # don't forget the dropout to prevent overfitting
        encoder_output = self.Encoder(X)
        encoder_output = tf.nn.dropout(encoder_output,0.1)
        for i in range(self.m-1):
            encoder_output = self.Encoder(encoder_output)
            encoder_output = tf.nn.dropout(encoder_output,0.1)
            
    def forward_pass(self, inputs):
        """
        do a forward pass using the transformer architecture and the Head then return the labels
        inputs :
        inputs (Tensor) ------------> tensor of shape [d,n] that represents a single sentence
        encoder_parameters(tuple)---> tuple of U_q,U_k,V,W,W1,W2,b1 and b2 respectively
        head_parameters (tuple)-----> tuple of W_c1 and b_c1 respectively
        output:
        y_hat (tensor)--------------> tesnor of size 6 represents predicted labels
        """
        X     = self.Transformer_block(inputs)
        # average all the states to feed it into the head
        X     = tf.reshape(tf.reduce_mean(X, axis = 1,),[-1,1]) 
        y_hat = self.Head(X)
        return y_hat
               
    def predict(self, X_batches):
        """
        predict the outputs given the inputs 
        input :
        X_batches (list)-------> list of tensors each tensor has size [d,n]
        output:
        Y_hat     (Tensor)-----> Tensor of outputs has size [batch_size,6]
        """
        # define list to store all Y_hat values to concatenate them later
        Y_hat = []
        # loop over each tensor 
        for batch in X_batches:
            # get the y_hat given that batch
            y_hat = self.forward_pass(batch)
            Y_hat.append(tf.transpose(y_hat))
        # concatenate all the predicted values 
        Y_hat = tf.concat(Y_hat, axis = 0)
        return Y_hat
     
    def evaluate(self, X_batches, Y, threshold = 0.7):
        """
        evaluate the model with the current parameters 
        inputs:
        X_batches (list)-------> list of tensors each tensor has size [d,n]
        Y         (Tensor)-----> true labels from the data 
        parameters (tuple)-----> tuple contain encoder parameters and Head parameters respectively
        threshold  (float)-----> the threshold to approximate the probabilities from y_hat
        """
        Y_hat = self.predict(X_batches)
        # calculate the loss between the true y and the predicted y
        loss  = tf.compat.v2.losses.binary_crossentropy(Y, Y_hat)
        average_loss = tf.reduce_mean(loss)
        # calculate the accuarcy between the true y and the predicted y
        # don't forget to approximate the probabilities using the threshold
        Y_hat = tf.where(Y_hat > threshold, 1,0)
        acc = tf.metrics.Accuracy()
        acc.update_state(Y, Y_hat)
        average_acc = acc.result().numpy()

        return average_loss,average_acc
    def get_batch(self, df_text, df_labels, st, batch_size):
        """
        this function deals with datafram directly
        inputs:
        df_text   (datafram)-------> datafram contains text comments
        df_labels (datafram)-------> datafram contains all the labels
        st        (int)------------> start index of the batch
        batch_size(int)------------> number of batchs
        output:
        X_batces,Y_batches(tuple)--> batches of embeddings and labels
        """
        # access the batches then turn them to numpy array
        sentences = df_text.iloc[st:st+batch_size].values
        X_batces = []
        # store the embeddings of each sentence in a list 
        for sentence in sentences:
            x = self.input_embeddings(sentence)
            # x dim is 300 x 50
            # we needto stack or concat them
            X_batces.append(x)
        Y_batches = df_labels.iloc[st:st+batch_size,:].values
        # note X_batches is a list
        return X_batces,Y_batches
    def train(self, text, labels, epochs, alpha, b, stop_early):
        """
        train the model given the data
        inputs :
        text       (datafram)-----> datafram column contains tokenized sentence 
        labels     (datafram)-----> datafram columns contain all the labels
        epochs     (int)----------> number of epochs to train the model
        b          (int)----------> number of batchs
        stop_early (tuple)--------> accuracy,loss thresholds to stop training
        output:
        parameters (tuple)--------> tuple of all the parameters of the encoder and the head after training 
        """
        # define the steps required to train all the batches
        steps = math.floor(len(text)/b)
        # define an adam optimizer to update the parameters
        # clip norm is used to avoid gradients explosion
        opt  = tf.compat.v2.optimizers.Adam(learning_rate = alpha,clipnorm = 1.0)
        # Define a list of accuracies calculated at each step and then averaged at the end  of each epoch
        accu_per_epoch = [0]
        for epoch in range(epochs) :
            # stop if we have reached desired accurcy
            if (np.mean(accu_per_epoch) > stop_early[0]) and (np.mean(loss_per_epoch) < stop_early[1]):
                break
            loss_per_epoch = []
            accu_per_epoch = []
            s = 1
            # loop and get a batch then update the parameters based on the loss 
            for st in range(0,len(text),b):
                X_batches,Y = self.get_batch(text, labels, st, b)
                # gradient tape is very effective method to get the gradients of the parameters
                with tf.GradientTape() as tape:
                    tape.watch(self.parameters)
                    loss,accu   = self.evaluate(X_batches, Y)
                    accu_per_epoch.append(accu)
                    loss_per_epoch.append(loss)

                gradients = tape.gradient(loss, self.parameters)
                # update the model using adam optimizer with the gradients
                opt.apply_gradients(zip(gradients, self.parameters))
                # update and print the results at each step
                sys.stdout.write("\r"+"Epoch : "+str(epoch+1)+\
                                 "  step : "+str(s)+"\\"+str(steps)+\
                                 "  loss : "+str(np.mean(loss_per_epoch))+\
                                 "  accuracy : "+str(np.mean(accu_per_epoch)))
                s += 1
            print("")
    def test(self, text, labels, b):
        # test function works the same way as train  except it doesn't update the parameters 
        steps = math.floor(len(text)/b)
        loss_per_epoch = []
        accu_per_epoch = []
        s = 1
        for st in range(0,len(text),b):
            X_batches,Y = self.get_batch(text, labels, st, b)    
            loss,accu   = self.evaluate(X_batches, Y)
            accu_per_epoch.append(accu)
            loss_per_epoch.append(loss)
            sys.stdout.write("\r"+"Epoch : 1"+\
                             "  step : "+str(s)+"\\"+str(steps)+\
                             "  test_loss : "+str(np.mean(loss_per_epoch))+\
                             "  test_accuracy : "+str(np.mean(accu_per_epoch)))
            s += 1
            


In [ ]:
model = Model(dic)

In [ ]:
model.import_parameters("parameters.pk1")

In [ ]:
# shuffle the data before dividing it into train and test
df = df.sample(frac = 1, random_state = 1).reset_index(drop=True)
# define a splitter to split the data into 60% training and the reset for testing
s = int(len(df)*0.6) 

In [ ]:
model.test(df.iloc[s:,1],df.iloc[s:,2:],256)